In [ ]:
!pip install chatterbot
!pip install chatterbot_corpus

     |████████████████████████████████| 63 kB 787 kB/s 
     |████████████████████████████████| 1.3 MB 16.5 MB/s 
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.32
    Uninstalling SQLAlchemy-1.4.32:
      Successfully uninstalled SQLAlchemy-1.4.32
     |████████████████████████████████| 117 kB 6.2 MB/s 


In [ ]:
!pip install pyngrok
!pip install flask
!pip install pyngrok
!pip install flask-ngrok

     |████████████████████████████████| 745 kB 13.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=40d182d921057719608ab46af277f1bfe885f34450dee921de9979c530d20537
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from tensorflow.python.framework.sparse_tensor import convert_to_tensor_or_sparse_tensor
import json
import pickle
import random
import nltk
import numpy
#import NewQeustion
#import JapaneseChatbot
#input the chatbot library
from chatterbot import ChatBot
#input the chatterbot trainers 
from chatterbot.trainers import ChatterBotCorpusTrainer

#imprt the nltk function and libraries
from nltk.stem import LancasterStemmer
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_yaml

#downlaod a punkt datase 
nltk.download('punkt')

#set a chatbot 
Aoi_bot = ChatBot('Aoi')
trainer = ChatterBotCorpusTrainer(Aoi_bot)
trainer.train("chatterbot.corpus.japanese")

#this is stemming process for the natural language process
stemming_process = LancasterStemmer()


#open the json file for the chatbot 
with open("/English_intents.json", encoding="utf-8") as input_data:
   content = json.load(input_data)

#open a chatbot.pickle for the learning 
try:
    with open("chatbot.pickle", "rb") as chatbot_data:
        tokens_character, catergory, training, output = pickle.load(chatbot_data)

except:
    #take the sourcefrom the 
    tokens_character = []
    catergory = []
    words_seperate = []
    tags_in_doc = []


    #for intent in data["intents"]:
    for intent in content["english_intents"]:
        for User_Input_pattern in intent["input_patterns"]:
            #tokenization 
            #using nlt and word tokenization to test the input
            characters = nltk.word_tokenize(User_Input_pattern)
            #segmentation 
            tokens_character.extend(characters)
            words_seperate.append(characters)
            tags_in_doc.append(intent["tag"])


        if intent["tag"] not in catergory:
            intent_tag = intent["tag"]
            catergory.append(intent_tag)


    token_words = [stemming_process.stem(token_word.lower()) for token_word in tokens_character if (token_word != "?" or token_word != "." or token_word != "!") ]

    word_set_list= set(token_words)
    token_words = list(word_set_list)
    token_words = sorted(token_words)
    print(token_words)
    token_labels = sorted(catergory)

    data_training = []
    data_output = []

    #find the total length / size of token labels. 
    num = len(token_labels)
    init = 0
    data_empty = [init for _ in range(num)]
   #create a bag of word list to store the words which is seperated 
    for i, tok in enumerate(words_seperate):
        bag_of_words_list = []
        check = False 
        match_words = [stemming_process.stem(j.lower()) for j in tok]

        for index_find_word in token_words:
            if index_find_word in match_words:
                bag_of_words_list.append(1)
            else:
                bag_of_words_list.append(0)

        output_data_rows = data_empty[:]
        tags_doc = tags_in_doc[i]
        equal = 1 
        output_data_rows[token_labels.index(tags_doc)] = equal

        data_training.append(bag_of_words_list)

        data_output.append(output_data_rows)

    output_training = numpy.array(data_training)
    data_output = numpy.array(data_output)
    print(data_output)

    with open("chatbot.pickle", "wb") as wb_pickle:
        pickle.dump((token_words, token_labels, output_training, data_output), wb_pickle)

try:
    dataset_collected = open('chatbotmodel.yaml', 'r')
    loaded_dataset_collected = dataset_collected.read()
    dataset_collected.close()
    AoiModel = model_from_json(loaded_dataset_collected)

    #myChatModel.load_weights("chatbotmodel.h5")
    AoiModel.load_weights("chatbotmodel.h5")
    print("Finish the loaded model")
    #print("Loaded model from disk")

except:
    # Make a neural network

    # Create a Aoi neural networ first
    AoiModel = Sequential()
    # Create hidden layer with 8, and the shape of imput data
    AoiModel.add(Dense(15, input_shape=[len(words)], activation='relu'))
    AoiModel.add(Dense(len(labels), activation='softmax'))

    # training the model 
    AoiModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      # training the model 
    AoiModel.fit(training, output, epochs=1000, batch_size=8)

    # write into a model and use 
    model_yaml = AoiModel.to_json()
    with open("chatbotmodel.yaml", "w") as y_file:
        y_file.write(model_yaml)

       # serialize weights to HDF5
    AoiModel.save_weights("chatbotmodel.h5")
    print("Saved model from disk")


def bag_of_words(inputdata, tokens_character):
    total = len(tokens_character)
    init = 0
    bag_list = [init for _ in range(total)]

    words_tokeniz = nltk.word_tokenize(inputdata)
    input_tokenzise = [stemming_process.stem(token.lower()) for token in words_tokeniz]

    for token in input_tokenzise:
        for i, j in enumerate(tokens_character):
            if j == token:
                 bag_list[i] = 1

    return numpy.array(bag_list)


def chatWithBot(inputText):
    Text_input_from_user=bag_of_words(inputText, tokens_character)
    Text_input_from_user_array = [Text_input_from_user]
    Text_input_from_user_text = numpy.array(Text_input_from_user_array)

    result = AoiModel.predict(Text_input_from_user_text[0:1])
    result_index = numpy.argmax(result)
   

    if inputText.upper() == "QUIT" or inputText.upper() == "QUIT.":
        return "Quited"
    if inputText.upper() == "A":
        return "Start on Basic Learn."
    elif inputText.upper() == "B":
        return "Start on Advanced Learn."
    elif inputText.upper() == "C":
        #JapaneseChatbot.Aoi_talk("你好")
        #response = Aoi_bot.get_response("你好")
        response = "こんにちは"
        return response
        #return "Start on Advanced II Learn."
    elif inputText.upper() == "D":
        return "Start Sangaku Lern."
    else:
        if numpy.all((numpyCurrentText == 0)):
            return "I did not get your meaning , try again"

        if result[0][result_index] > 0.7:
            for tg in data["english_intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            return random.choice(responses)




    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Training botprofile.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training psychologcial_situation.yml: [####################] 100%
Training season.yml: [####################] 100%
Training shopping.yml: [####################] 100%
Training sport.yml: [####################] 100%
Training travel.yml: [####################] 100%
Epoch 1/1000
20/20 [==============================] - 1s 2ms/step - loss: 3.6504 - accuracy: 0.0190 
Epoch 2/1000
20/20 [==============================] - 0s 2ms/step - loss: 3.6175 - accuracy: 0.0253
Epoch 3/1000
20/20 [==============================] - 0s 2ms/step - loss: 3.5879 - accuracy: 0.0633
Epoch 4/1000
20/20

In [ ]:
def JPchatwithAoi(inputText):
  response = Aoi_bot.get_response(inputText)
  return response

In [ ]:
BotRresponses = "hello"

print(chatWithBot(BotRresponses))

Hello! Welcome to the chatroom. Let's learn Japanese happily together! 
 Option A: Start from Basic 50 Japanese words.Option B: Start from Learning Japanese vocabularies. Option C: Start from simple conversation. Option D: Try the problem set of Sangaku.


In [ ]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__)
run_with_ngrok(app)
count=0

@app.route("/",methods=['GET', 'POST'])
def hello1():
  return "Hello Geeks!! from Google Colab"


@app.route("/chatroom",methods=['GET', 'POST'])
def hello():
  
  BotRresponses = request.form['chatInput']
  if BotRresponses.upper()=='START' or BotRresponses.upper()=='EXIT' :
    return jsonify(AoiReply='Hello! Welcome to Learn Japanese with Aoi! \n A: Start from Basic 50 Japanese words.\n B: Start from Learning Japanese vocabularies.\n C: Start from simple conversation.\n D: Try the problem set of Sangaku.')
  elif BotRresponses=='D':
     return "な \n A: a \n B: na \n C: ni"
  else:
    #BotRresponses = request.form['chatInput']
    return jsonify(AoiReply=chatWithBot(BotRresponses))
    #return "Hello Geeks!! from Google Colab"

@app.route("/JPconversation",methods=['GET', 'POST'])
def JPCon():
  
  BotRresponses = request.form['chatInput']
  if BotRresponses.upper()=="C":
    return jsonify(AoiReply='こんにちは,わたしわAoiです.')
  else:
    print(BotRresponses)
    Answer=str(JPchatwithAoi(BotRresponses))
    #print(Answer)
    return jsonify(AoiReply=Answer)

@app.route("/BasicJP",methods=['GET', 'POST'])
def StartBasicLearnng():
  
  BotRresponses = request.form['chatInput']
  if BotRresponses.upper()=="A":
    Answer=str(BasicLearnng())
    #print(Answer)
    return jsonify(AoiReply=Answer)
  else:
    Answer=str(BasicLearnng())
    #print(Answer)
    return jsonify(AoiReply=Answer)


@app.route("/AdvancedJP",methods=['GET', 'POST'])
def StartAdvancedLearning():
  
  BotRresponses = request.form['chatInput']
  if BotRresponses.upper()=="B":
    Answer=str(AdvancedLearning())
    #print(Answer)
    return jsonify(AoiReply=Answer)
  else:
    Answer=str(AdvancedLearning())
    #print(Answer)
    return jsonify(AoiReply=Answer)

@app.route("/Sangaku",methods=['GET', 'POST'])
def StartAdvancedIILearning():
  
  BotRresponses = request.form['chatInput']
  if BotRresponses.upper()=="D":
    Answer = str(AdvancedIILearning("null"))
    print(Answer)
    return jsonify(AoiReply=Answer)
  else: 
    Answer = str(AdvancedIILearning(BotRresponses))
    print(Answer)
    return jsonify(AoiReply=Answer)




@app.route("/image_upload",methods=['GET', 'POST'])
def SangakuUpload_link():
  
  #BotRresponses = request.form['chatInput']
  #if BotRresponses =="B":
 
  encodeimage = request.form['image']
  print(encodeimage)
  SangakuUpload(encodeimage)
  return "upload_success"
   
if __name__ == "__main__":
  count=0
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ac0e-34-91-110-130.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Apr/2022 07:30:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2022 07:30:23] "GET /favicon.ico HTTP/1.1" 404 -


['Sangaku', 'Advanced', 'Basic']
11
20
7
19
{'_id': 11, 'Hiragana': 'し', 'option': 'shi'}
し
shi


...

おなまえは


127.0.0.1 - - [09/Apr/2022 07:46:25] "POST /chatroom HTTP/1.1" 200 -


MongoClient(host=['cluster0-shard-00-01.f6l9p.mongodb.net:27017', 'cluster0-shard-00-02.f6l9p.mongodb.net:27017', 'cluster0-shard-00-00.f6l9p.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, ssl=True, replicaset='atlas-m289d7-shard-0', authsource='admin', retrywrites=True, w='majority')
['Japanese', 'Level_Indicator', 'Sangaku', 'User', 'admin', 'local']
['JP_Sangaku']


127.0.0.1 - - [09/Apr/2022 07:46:31] "POST /Sangaku HTTP/1.1" 200 -


{'_id': 1, 'EN_QUESTION': 'two circles of radius r are tangent to\neach other and touch the sector internally. A small circle of radius t touches both the sector and a chord of length d. If d = 3.62438 and 2t = 0.34, find 2r', 'JP_Question': '"半径rの2つの円はに接しています\nお互いに、そして内部でセクターに触れます。 半径tの小さな円は、扇形と長さdの弦の両方に接しています。 d=3.62438および2t=0.34の場合、2rを見つけます"', 'Ans': '2r = 3.025', 'URL': 'https://firebasestorage.googleapis.com/v0/b/japaneseapp-8816d.appspot.com/o/%E6%9C%AA%E5%91%BD%E5%90%8D.png?alt=media&token=7dd3a45d-0f0b-4f74-8d0a-10804d628af2'}
two circles of radius r are tangent to
each other and touch the sector internally. A small circle of radius t touches both the sector and a chord of length d. If d = 3.62438 and 2t = 0.34, find 2r
Please try the following Sangaku.
<https://firebasestorage.googleapis.com/v0/b/japaneseapp-8816d.appspot.com/o/%E6%9C%AA%E5%91%BD%E5%90%8D.png?alt=media&token=7dd3a45d-0f0b-4f74-8d0a-10804d628af2>
"半径rの2つの円はに接しています
お互いに、そして内部でセクターに触れます。 半径tの小さな円は、扇形と長さdの弦の両方に接して

In [ ]:
!pip3 install pymongo[srv]
!pip3 install pymongo[tls]
!pip install dnspython
!pip install pymongo[srv]
!pip3 install 'pymongo[srv]'

     |████████████████████████████████| 269 kB 13.0 MB/s 


In [ ]:
!pip install dnspython
!pip install pymongo

In [ ]:
! python -m pip install pymongo==3.7.2

     |████████████████████████████████| 406 kB 13.6 MB/s 
  Attempting uninstall: pymongo
    Found existing installation: pymongo 4.0.2
    Uninstalling pymongo-4.0.2:
      Successfully uninstalled pymongo-4.0.2


Japanese Basic Learning 

In [ ]:
import pymongo
import json
from array import *
from pymongo import MongoClient

# generate random integer values
from random import seed
from random import randint


def BasicLearnng():
  cluster = ""
  client = MongoClient(cluster)
  client.stats
  #client.list_database_names()
  #client.list_database_names()
  #print(client.list_database_names())
  db = client.Japanese
  print(db.list_collection_names())


  random_num=randint(1, 45)
  print(random_num)
  other_option=[]


  for x in range(3):
    other_option.append(randint(0, 44))
    print(other_option[x])



      
  outcome=db.Basic
  result1 = outcome.find_one({'_id':random_num})
  print(result1)


  Higara1 = result1['Hiragana']
  Answer1 = result1['option']
  print(Higara1)
  print(Answer1)

  ans1 = other_option[0]
  ans2 = other_option[1]
  ans3 = other_option[2]


  Answer_A = outcome.find_one({'_id':ans1})
  Answer_B = outcome.find_one({'_id':ans2})
  Answer_D = outcome.find_one({'_id':ans3})

  Ans_A =Answer_A['option']
  Ans_B =Answer_B['option']
  Ans_D =Answer_D['option']
  
  print(Ans_A)
  print(Ans_B)
  print(Ans_D)


  Full= "Please find out the correct answer.\n" +Higara1 + "\n" + "A:" + Answer1 +"\n" + "B:" + Ans_B +"\n"+ 'C:'+ Ans_A  +"\n"+ 'D:'+ Ans_D +"\n" 
  return Full
  #print(hiragara[['id']['1']])


  #json_object = json.loads(hiragara)
  #Ans=json_object["option"]
  #print(Ans)
  #for x in outcome.find():
      #print(x)

Show the AdvancedI question bank from the database

In [ ]:
import pymongo
import json
from array import *
from pymongo import MongoClient

# generate random integer values
from random import seed
from random import randint

def AdvancedLearning():
  link = ""
  client = MongoClient(link)
  client.stats
  #client.list_database_names()
  #client.list_database_names()
  #print(client.list_database_names())
  db = client.Japanese
  print(db.list_collection_names())


  random_num=randint(1, 45)
  print(random_num)
  other_option=[]


  for x in range(3):
    other_option.append(randint(0, 71))
    print(other_option[x])



      
  outcome=db.Advanced
  result1 = outcome.find_one({'_id':random_num})
  print(result1)


  Higara1 = result1['Hiragana']
  Answer1 = result1['English']
  print(Higara1)
  print(Answer1)

  ans1 = other_option[0]
  ans2 = other_option[1]
  ans3 = other_option[2]


  Answer_A = outcome.find_one({'_id':ans1})
  Answer_B = outcome.find_one({'_id':ans2})
  Answer_D = outcome.find_one({'_id':ans3})


  Ans_A =Answer_A['English']
  Ans_B =Answer_B['English']
  Ans_D =Answer_D['English']
  
  print(Ans_A)
  print(Ans_B)
  print(Ans_D)

  Full= "Please find out the correct answer.\n" +Higara1 + "\n" + "A:" + Ans_A +"\n" + "B:" + Ans_B +"\n"+ 'C:'+ Answer1 +"\n"+ 'D:'+ Ans_D +"\n" 
  return Full

Show the advanced II question bank from the database


In [ ]:
import pymongo
import json
from array import *
from pymongo import MongoClient

# generate random integer values
from random import seed
from random import randint

def AdvancedIILearning(userinput): 
  
  link = ""
  client = MongoClient(link)
  print(client)

  client.stats
  #client.list_database_names()
  #client.list_database_names()
  print(client.list_database_names())
  db = client.Sangaku
  print(db.list_collection_names())
  outcome=db.JP_Sangaku

  if (userinput.upper()!="NEXT"): 
      
    result1 = outcome.find_one({'_id':1})

    print(result1)
    EN_Question = result1['EN_QUESTION']
    print( EN_Question )
    JP_Question = result1['JP_Question']
    ANS = result1['Ans']
    URL=result1['URL']

    Answer_A = "t = 69.75494"
    Answer_B = "2t = 19.854"
    Answer_C = "d = 39.7494"
    Answer_D = "2r = 3.025"

  else:
    result1 = outcome.find_one({'_id':2})

    print(result1)
    EN_Question = result1['EN_QUESTION']
    print( EN_Question )
    JP_Question = result1['JP_Question']
    ANS = result1['Ans']
    URL=result1['URL']

    Answer_A = "r/2 = 3.025"
    Answer_B = "2r = 19"
    Answer_C = "r=9"
    Answer_D = "r=(9-5sqrt(3))"


  
  
  
  

  Full= "Please try the following Sangaku.\n" + "<" + URL + ">" +"\n" + JP_Question +"\n"+  EN_Question+"\n" +  "A:" + Answer_A +"\n"+ "B:" + Answer_B +"\n"+ "C:" +Answer_C +"\n"+ "D:" + Answer_D+"\n"
  print(Full)

  return Full
  

Sangaku Upload function

In [ ]:
import pymongo
import json
from array import *
import gridfs
from pymongo import MongoClient
import io
from PIL import Image

# generate random integer values
from random import seed
from random import randint

def SangakuUpload(encodeimage):
  link = ""
  client = MongoClient(link)
  client.stats
  #client.list_database_names()
  #client.list_database_names()
  #print(client.list_database_names())
  db = client.Japanese
  print(db.list_collection_names())
  outcome=db.Sangaku
  database = client['Sangaku']
  fs = gridfs.GridFS(database)


  with open(encodeimage, 'rb') as f:
    contents = f.read()

  fs.put(contents, filename="file")
  im=encodeimage
  image_bytes = io.BytesIO()
  im.save(image_bytes, format='JPEG')
  image = {
    'data': image_bytes.getvalue()
}

  image_id = image.insert_one(image).inserted_id

 


  